In [1]:
import torch as nn
import numpy as np
import pandas as pd

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

class SingleScoreCNN(nn.Module):
    def __init__(self):
        super(SingleScoreCNN, self).__init__()
        # Define the layers of the CNN
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 64 * 8)  # Adjust the size based on the input size
        self.fc2 = nn.Linear(64 * 8, 64 * 2)  
        self.fc3 = nn.Linear(64 * 2, 64) 
        self.fc4 = nn.Linear(64, 1)
    
    def forward(self, x):
        # Define the forward pass
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = F.tanh(self.fc3(x))
        x = self.fc4(x)
        return x

# Example usage:
model = SingleScoreCNN()
output = model(torch.randn(1, 3, 64, 64))  # Example input tensor (batch_size, channels, height, width)

# Define the loss and optimizer
criterion = nn.MSELoss()  # For example, if it's a regression task
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [13]:
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class MeasuresDataset(Dataset):
    def __init__(self, csv_file, image_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        page = str(int(self.data.iloc[idx]['Page']))
        label = self.data.iloc[idx]['Final Score']
        
        img_path = os.path.join(self.image_dir, f"{page}.jpg")
        # print(page, label, img_path)
        image = Image.open(img_path).convert('RGB')
        
        # Apply transformations, if provided
        if self.transform:
            image = self.transform(image)
        
        return image, torch.tensor(label, dtype=torch.float)


transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize to the size expected by the network
    transforms.ToTensor()
])

dataset = MeasuresDataset(csv_file='measures.csv', image_dir='./combined', transform=transform)
# dataset = MeasuresDataset(csv_file='measures.csv', image_dir='./', transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Iterate over the dataloader
# for images, labels in dataloader:
#     # Feed data to the model
#     output = model(images)
#     # Compute the loss, etc.
#     print(output, labels)

In [14]:
import torch
import torch.optim as optim
import torch.nn as nn

# Prepare model, loss function, and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for images, labels in dataloader:
        # Move data to GPU if available
        # images, labels = images.cuda(), labels.cuda() if torch.cuda.is_available() else (images, labels)
        
        optimizer.zero_grad()  # Zero the parameter gradients
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs.squeeze(), labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        # Accumulate the loss
        epoch_loss += loss.item()
    
    # Compute and print the average loss for the epoch
    avg_loss = epoch_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], MSE Loss: {avg_loss:.4f}")

FileNotFoundError: [Errno 2] No such file or directory: './combined\\53093.jpg'

In [5]:
torch.save(model,"simple_model.pth")

In [6]:
model = torch.load("simple_model.pth")

In [7]:
model

SingleScoreCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=4096, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=1, bias=True)
)

In [8]:
def model_predict_class(image):
    score = model(image)
    if score < 0.3:
        category = "Very Poor"
    elif score < 0.4:
        category = "Poor"
    elif score < 0.6:
        category = "Fair"
    elif score < 0.8:
        category = "Good"
    else:   
        category = "Very Good"
    return category       

In [15]:
for images, labels in dataloader:
    # Feed data to the model
    output = model_predict_class(images)
    # Compute the loss, etc.
    print(output, labels)

FileNotFoundError: [Errno 2] No such file or directory: './combined\\30801.jpg'